# El puente estrecho

**Alumno:** Axel Daniel Malváez Flores

**Semestre:** 5to Semestre

In [ ]:
import threading
import time
import multiprocessing as mp
import sys
import numpy as np


class Semaphore():
    def __init__(self,sentido=0):
        self.lock = threading.Condition() 
        self.sentido = sentido

    def up(self):
        with self.lock:
            while self.sentido < 0:
                self.lock.wait()
            self.sentido += 1
            self.lock.notify() 

    def down(self):
        with self.lock:
            while self.sentido > 0:
                self.lock.wait() 
            self.sentido -= 1
            self.lock.notify() 
            
class Puente():
    def __init__(self, semaforo):
        self.autos_permitidos = 3
        self.user = -1
        self.lock = threading.Condition()
        self.semaforo = semaforo

    def take(self, user, sentidoSTR, sentido):
        with self.lock:
            while (self.semaforo.sentido < 0 and sentido == 'derecho') or (self.semaforo.sentido > 0 and sentido == 'izquierdo') or (np.abs(sentido) == self.autos_permitidos):
                self.lock.wait()
            if sentido < 0:
                self.user = user
                print(f'--- Carro {user} de sentido izquierdo pasa por el puente ---\n')
                self.lock.notifyAll()
            elif sentido > 0:
                self.user = user
                print(f'--- Carro {user} de sentido derecho pasa por el puente ---\n')
                self.lock.notifyAll()
            else:
                self.user = user
                print(f'--- Carro {user} de sentido {sentidoSTR} pasa por el puente ---\n')
                self.lock.notifyAll()

    def drop(self, user, sentidoSTR, sentido):
        with self.lock:
            while self.semaforo.sentido < 0 and sentido > 0:
                self.lock.wait()
            if sentido < 0:
                self.user = user
                print(f'*** Carro {user} de sentido izquierdo pasó por el puente ***\n')
                self.lock.notifyAll()
            elif sentido > 0:
                self.user = user
                print(f'*** Carro {user} de sentido derecho pasó por el puente ***\n')
                self.lock.notifyAll()
            else:
                self.user = user
                print(f'***  Carro {user} de sentido {sentidoSTR} pasó por el puente ***\n')
                self.lock.notifyAll()
            

class Auto(threading.Thread):
    def __init__(self, identificador, sentido, puente, semaforo):
        threading.Thread.__init__(self)
        self.id = identificador
        self.sentido = sentido 
        self.puente = puente
        self.semaforo = semaforo
    
    def run(self):
        while True:
            if self.sentido == 'derecho':
                print(f'Carro {self.id} de sentido derecho quiere pasar\n')
                self.semaforo.up()            
                #time.sleep(1)
                self.puente.take(self.id, self.sentido, self.semaforo.sentido)
                #time.sleep(1)                
                self.puente.drop(self.id, self.sentido, self.semaforo.sentido)
                #time.sleep(1)
                time.sleep(1)
            else:
                print(f'Carro {self.id} de sentido izquierdo quiere pasar\n' )
                self.semaforo.down()
                #time.sleep(1)
                self.puente.take(self.id, self.sentido, self.semaforo.sentido)
                #time.sleep(1)                
                self.puente.drop(self.id, self.sentido, self.semaforo.sentido)
                #time.sleep(1)
                time.sleep(1)

if __name__ == '__main__':
    n_izq = 5
    n_der = 5

    c = [Puente(i) for i in range(n_izq + n_der)]
    
    semaforo = Semaphore()  # Su valor está en 3

    puente = Puente(semaforo)

    sys.stdout.write("\n")
    
    # Number, right/left, semaphore
    p_izq = [ Auto(i, 'izquierdo' , puente, semaforo) for i in range(n_izq) ]
    p_der = [ Auto(i, 'derecho', puente, semaforo) for i in range(n_der) ]

    for i in range(n_izq):
        p_izq[i].start()
        p_der[i].start()
        
    for i in range(n_izq):
        p_izq[i].join()
        p_der[i].join()